In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [27]:
df_obama = pd.read_csv('data/obama_cleaned.csv')
df_romney = pd.read_csv('data/romney_cleaned.csv')

In [22]:
df_romney.head()

,date,time,tweets,class
0,2012-10-16 00:00:00,09:38:08-05:00,insidiousmitt romneys bain helped philip morri...,-1
1,2012-10-16 00:00:00,10:14:18-05:00,you mean like romney cheated in primary,-1
2,2012-10-16 00:00:00,09:27:16-05:00,mitt romney still doesnt believe that we have ...,-1
3,2012-10-16 00:00:00,10:11:43-05:00,romneys tax plan deserves a 2nd look because h...,-1
4,2012-10-16 00:00:00,10:13:17-05:00,hope romney debate prepped w the same people a...,1


In [4]:
df_obama.head()

,date,time,tweets,class
0,2012-10-16 00:00:00,10:28:53-05:00,kirkpatrick who wore a baseball cap embroidere...,0
1,2012-10-16 00:00:00,10:04:30-05:00,obama debates that cracker ass cracker tonight...,1
2,2012-10-16 00:00:00,09:50:08-05:00,youre missing the point im afraid you do not u...,0
3,2012-10-16 00:00:00,10:00:16-05:00,i was raised as a democrat left the party year...,-1
4,2012-10-16 00:00:00,09:48:07-05:00,the obama camp cant afford to lower expectatio...,0


In [66]:
X_train_obama, X_test_obama, y_train_obama, y_test_obama = train_test_split(df_romney['tweets'], df_romney['class'], test_size=0.2, random_state=42)

In [ ]:
def vectorization(tweets_column):
    vectorizer = TfidfVectorizer()
    tweets_column = vectorizer.fit_transform(tweets_column)
    return vectorizer

In [67]:
obama_vectorizer = TfidfVectorizer()
tfidf_obama_train = obama_vectorizer.fit_transform(X_train_obama)
tfidf_obama_test = obama_vectorizer.transform(X_test_obama)


In [68]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(tfidf_obama_train, y_train_obama)

In [69]:
predictions = clf.predict(tfidf_obama_test)

In [70]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test_obama, predictions)

0.5172719220549159

In [71]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf = clf.fit(tfidf_obama_train, y_train_obama)

/Users/aarshpatel/Downloads/DMTM (CS 583)/Project 2/.venv/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [72]:
predictions = clf.predict(tfidf_obama_test)

In [73]:
accuracy_score(y_test_obama, predictions)

0.5615589016829052

: 